<a href="https://colab.research.google.com/github/prasadmlexpert/objectidentificationvideo/blob/main/YoloV5CustomTrainingBDD100k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/

/content


In [4]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 13162, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 13162 (delta 23), reused 26 (delta 13), pack-reused 13119
Receiving objects: 100% (13162/13162), 12.54 MiB | 29.52 MiB/s, done.
Resolving deltas: 100% (9034/9034), done.


In [5]:
pip install -r yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 5.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [16]:
%cd /content/drive/.shortcut-targets-by-id/16kphvlmJsL1izzffpLfadthz0YrraYkW/ImageIdetificationProject/Project_work/

/content/drive/.shortcut-targets-by-id/16kphvlmJsL1izzffpLfadthz0YrraYkW/ImageIdetificationProject/Project_work


In [17]:
!ls

 bdd100k				  images
 bdd100k_images_100k.rar		  labels
'Copy of YOLOv4-Roboflow-PyTorch.ipynb'   objectidentificationvideo
 data					  results.jpg
 dataset				  results.txt
 detections


In [18]:
!cp bdd100k_images_100k.rar /content

In [19]:
%cd /content

/content


In [20]:
!unrar x bdd100k_images_100k.rar

Streaming output truncated to the last 5000 lines.
Extracting  bdd100k_images_100k/val/c4756fa2-027007f8.txt                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-1393ce84.jpg                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-1393ce84.txt                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-d8a26f8c.jpg                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-d8a26f8c.txt                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-fb46c2d8.jpg                 97%  OK 
Extracting  bdd100k_images_100k/val/c4770a97-fb46c2d8.txt                 97%  OK 
Extracting  bdd100k_images_100k/val/c477c67b-6eb95776.jpg                 97%  OK 
Extracting  bdd100k_images_100k/val/c477c67b-6eb95776.txt                 97%  OK 
Extracting  bdd100k_images_100k/val/c477c67b-89221134.jpg                 97%  OK 
Extracting  bdd100k_images_10

In [21]:
!rm -rf bdd100k_images_100k.rar 

In [22]:
!mkdir images labels

mkdir: cannot create directory ‘images’: File exists
mkdir: cannot create directory ‘labels’: File exists


In [23]:
!mkdir images/train images/val images/test labels/train labels/val labels/test

mkdir: cannot create directory ‘images/train’: File exists
mkdir: cannot create directory ‘images/val’: File exists
mkdir: cannot create directory ‘images/test’: File exists
mkdir: cannot create directory ‘labels/train’: File exists
mkdir: cannot create directory ‘labels/val’: File exists
mkdir: cannot create directory ‘labels/test’: File exists


In [24]:
train_images = [os.path.join('/content/bdd100k_images_100k/train', x) for x in os.listdir('/content/bdd100k_images_100k/train') if x[-3:] == "jpg"]
train_annotations = [os.path.join('/content/bdd100k_images_100k/train', x) for x in os.listdir('/content/bdd100k_images_100k/train') if x[-3:] == "txt"]

In [25]:
val_images = [os.path.join('/content/bdd100k_images_100k/val', x) for x in os.listdir('/content/bdd100k_images_100k/val') if x[-3:] == "jpg"]
val_annotations = [os.path.join('/content/bdd100k_images_100k/val', x) for x in os.listdir('/content/bdd100k_images_100k/val') if x[-3:] == "txt"]

In [26]:
test_images = [os.path.join('/content/bdd100k_images_100k/test', x) for x in os.listdir('/content/bdd100k_images_100k/test') if x[-3:] == "jpg"]

In [27]:
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train/')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'labels/train/')
move_files_to_folder(val_annotations, 'labels/val/')

In [28]:
rm -rf bdd100k_images_100k

In [29]:
!ls

drive  images  labels  sample_data  yolov5


In [3]:
%cd /content/yolov5/
!ls

/content/yolov5
CONTRIBUTING.md  hubconf.py   README.md		train.py	yolov5n.pt
data		 LICENSE      requirements.txt	tutorial.ipynb	yolov5xbdd.pt
detect.py	 models       runs		utils
export.py	 __pycache__  setup.cfg		val.py


In [32]:
!wget https://raw.githubusercontent.com/prasadmlexpert/objectidentificationvideo/main/objectidentification.yml -P /content/yolov5/data/

--2022-07-29 00:45:33--  https://raw.githubusercontent.com/prasadmlexpert/objectidentificationvideo/main/objectidentification.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284 [text/plain]
Saving to: ‘/content/yolov5/data/objectidentification.yml’

objectidentificatio 100%[===================>]     284  --.-KB/s    in 0s      

2022-07-29 00:45:33 (11.4 MB/s) - ‘/content/yolov5/data/objectidentification.yml’ saved [284/284]



In [33]:
!cat /content/yolov5/data/objectidentification.yml

train: ../images/train/
val: ../images/val/
test: ../images/test/

#number of classes
nc: 14

#class names
names: ['car', 'truck', 'bus', 'train', 'person', 'rider', 'traffic sign', 'traffic light', 'motorcycle', 'pedestrian', 'bicycle', 'other vehicle', 'trailer', 'other person']




In [38]:
!rm -rf /content/yolov5/runs/train/exp

In [43]:
!cp /content/drive/.shortcut-targets-by-id/16kphvlmJsL1izzffpLfadthz0YrraYkW/ImageIdetificationProject/Project_work/data/yolov5xbdd.pt /content/yolov5/

In [44]:
!ls

CONTRIBUTING.md  hubconf.py   README.md		train.py	yolov5xbdd.pt
data		 LICENSE      requirements.txt	tutorial.ipynb
detect.py	 models       runs		utils
export.py	 __pycache__  setup.cfg		val.py


In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 156 kB 58.3 MB/s 
     |████████████████████████████████| 181 kB 45.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=3ce31a61e58ec2af8836f7a9d09a8f4830c922374843c3f38cebed578a39f508
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
!python train.py --img 640 --batch 16 --epochs 4 --data objectidentification.yml --weights yolov5xbdd.pt

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 1
wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
train: weights=yolov5xbdd.pt, cfg=, data=objectidentification.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=4, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1,

In [ ]:
!cp /content/yolov5/runs/train/exp3/weights/best.pt /content/drive/.shortcut-targets-by-id/16kphvlmJsL1izzffpLfadthz0YrraYkW/ImageIdetificationProject/Project_work/data/yolov5xbdd.pt